In [1]:
import os
import csv
import re

# Define the directory containing the files
directory = "metric_results"

# Define the output CSV file
output_csv = "statistics.csv"

# Open the output CSV file for writing
with open(output_csv, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header row
    csv_writer.writerow(["count", "temperature", "values"])

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            # Extract the count and temperature from the filename using regex
            match = re.search(r'rag-file-count-(\d+)-gpt-4o-temp-([0-9.]+)\.txt', filename)
            if match:
                count = int(match.group(1))
                temperature = float(match.group(2))

                # Read the data from the file
                with open(os.path.join(directory, filename), 'r') as file:
                    for line in file:
                        # Convert the line to a list of floats
                        values = eval(line.strip())
                        # Write the data to the CSV
                        csv_writer.writerow([count, temperature, values])

In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols

In [5]:
data = pd.read_csv("ANOVA.csv")

In [6]:
data.head()

,count,temperature,P1
0,0,0.5,0.025974
1,0,0.5,0.000000
2,0,0.5,0.025974
3,0,0.5,0.077922
4,0,0.5,0.051948


In [7]:
data['P1'] = pd.to_numeric(data['P1'])

In [8]:
data['temperature'] = data['temperature'].astype(str)

In [9]:
data['count'] = data['count'].astype(str)

In [10]:
model = ols('P1 ~ temperature + count', data=data).fit()

In [11]:
anova_table = sm.stats.anova_lm(model, typ=2)

In [12]:
print(anova_table)

               sum_sq     df          F        PR(>F)
temperature  0.049449    4.0   1.229368  2.980819e-01
count        2.984394    6.0  49.464062  1.647666e-43
Residual     3.408905  339.0        NaN           NaN


In [13]:
tukey_temp = pairwise_tukeyhsd(data['P1'], data['temperature'])

In [15]:
print(tukey_temp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
   0.5   0.75   0.0108 0.9902 -0.0523 0.0739  False
   0.5    1.0   -0.018 0.9357 -0.0811 0.0451  False
   0.5   1.25  -0.0202 0.9046 -0.0833 0.0429  False
   0.5    1.5  -0.0145 0.9703 -0.0776 0.0486  False
  0.75    1.0  -0.0288 0.7221 -0.0919 0.0343  False
  0.75   1.25   -0.031 0.6623 -0.0941 0.0321  False
  0.75    1.5  -0.0252 0.8083 -0.0883 0.0379  False
   1.0   1.25  -0.0022    1.0 -0.0653 0.0609  False
   1.0    1.5   0.0035 0.9999 -0.0596 0.0666  False
  1.25    1.5   0.0058 0.9991 -0.0573 0.0689  False
---------------------------------------------------


In [16]:
tukey_count = pairwise_tukeyhsd(data['P1'], data['count'])

In [17]:
print(tukey_count)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     0      1   0.3153    0.0  0.2558  0.3749   True
     0     10   0.2496    0.0    0.19  0.3092   True
     0      2   0.1301    0.0  0.0706  0.1897   True
     0      3   0.1382    0.0  0.0786  0.1977   True
     0      4   0.1844    0.0  0.1249   0.244   True
     0      5   0.1943    0.0  0.1347  0.2538   True
     1     10  -0.0657   0.02 -0.1253 -0.0062   True
     1      2  -0.1852    0.0 -0.2448 -0.1256   True
     1      3  -0.1771    0.0 -0.2367 -0.1176   True
     1      4  -0.1309    0.0 -0.1905 -0.0713   True
     1      5   -0.121    0.0 -0.1806 -0.0615   True
    10      2  -0.1195    0.0  -0.179 -0.0599   True
    10      3  -0.1114    0.0  -0.171 -0.0519   True
    10      4  -0.0652 0.0217 -0.1248 -0.0056   True
    10      5  -0.0553 0.0882 -0.1149  0.0042  False
     2      3   0.0081 0.9997 -0.0515  0.0676 